In [1]:
import matplotlib.pyplot as plt
import scipy.constants as cons
import pandas as pd
import numpy as np
import scipy
import cmath
import time
import os

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from qutip import *
from funcs import *

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


In [84]:
# Reading in data from file
df1 = pd.read_csv('cubic_good2.csv')
df1 = df1[df1.nmean < 6.05].sort_values('nmean')

df2 = pd.read_csv('inner_cubic_good4.csv')
df2 = df2[df2.nmean < 6.05].sort_values('nmean')

In [85]:
df1.describe()

,Wnorm,WLN,gamma,r,nmean,Ndim,xcount,ycount,time1,time2
count,1798.000000,1798.000000,1798.000000,1798.000000,1798.000000,1798.0,1798.0,1798.0,1798.000000,1798.000000
mean,0.999981,0.529135,0.356310,0.347195,2.065693,150.0,200.0,200.0,2.107560,4.906542
std,0.000031,0.334578,0.264242,0.262180,1.702465,0.0,0.0,0.0,1.796709,0.095753
min,0.999674,-0.000004,0.000000,0.000000,0.000000,150.0,200.0,200.0,1.128302,4.823785
25%,0.999977,0.245125,0.139241,0.128205,0.579873,150.0,200.0,200.0,1.160968,4.882004
50%,0.999994,0.542907,0.303797,0.282051,1.619368,150.0,200.0,200.0,1.181727,4.897351
75%,0.999998,0.802308,0.544304,0.512821,3.305779,150.0,200.0,200.0,2.318366,4.913276
max,1.000000,1.253392,1.000000,1.000000,6.045843,150.0,200.0,200.0,11.643558,6.636036


In [86]:
# Reading out relavent columns
WLNs1 = np.array(df1.WLN)
WLNs2 = np.array(df2.WLN)

nmeans1 = np.array(df1.nmean)
nmeans2 = np.array(df2.nmean)

sqf1 = np.array(df1.r)
sqf2 = np.array(df2.r)

gamma1 = np.array(df1.gamma)
gamma2 = np.array(df2.gamma)

In [89]:
fig1, ax1 = my3dscatter(gamma1, sqf1, nmeans1, title='', axeslabels=['gamma', 'r', 'WLN'])

plt.show()

In [6]:
fig2, ax2 = my3dscatter(gamma2, sqf2, WLNs2, title='', axeslabels=['gamma', 'r', 'WLN'])

plt.show()

In [72]:
# Creating bins for nmean data
bins1 = np.arange(-0.05, 6.1, 0.1)
bins2 = np.arange(-0.05, 6.1, 0.1)

df1['bins'] = pd.cut(df1.nmean, bins1)
df2['bins'] = pd.cut(df2.nmean, bins2)

In [73]:
maxWLNs1 = []
for bin in np.unique(df1.bins):
    maxWLNs1.append(df1[df1.bins == bin].WLN.max())

In [74]:
maxWLNs2 = []
for bin in np.unique(df2.bins):
    maxWLNs2.append(df2[df2.bins == bin].WLN.max())

In [75]:
df1max = df1[df1.WLN.isin(maxWLNs1)]
df2max = df2[df2.WLN.isin(maxWLNs2)]

In [76]:
# Generate Data for Fock states
fnmeans = []
fWLNs = []
N = 10
for i in range(1,7):
    fockstate = fock(N, i)
    fnmeans.append(expect(num(N), fockstate))
    fWLNs.append(wln(fockstate, 1e-5)[0])

In [77]:
plt.figure()
plt.plot(df1max.nmean, df1max.WLN)
plt.plot(df2max.nmean, df2max.WLN)
plt.scatter(fnmeans, fWLNs)
plt.show()

In [27]:
df2max

,Wnorm,WLN,gamma,r,nmean,Ndim,xbound,xcount,ybound,ycount,time1,time2,bins
5,0.999999,0.004390,0.051282,0.000000,0.049739,150,"[-5, 6]",200,"[-6, 5]",200,2.324375,4.872450,"(-0.05, 0.05]"
48,0.999998,0.067681,0.082051,0.051282,0.143542,150,"[-8, 9]",200,"[-9, 6]",200,2.405482,4.940140,"(0.05, 0.15]"
50,0.999988,0.170235,0.102564,0.051282,0.245506,150,"[-10, 11]",200,"[-11, 11]",200,2.319819,4.921960,"(0.15, 0.25]"
91,0.999963,0.251516,0.112821,0.102564,0.340673,150,"[-11, 12]",200,"[-12, 11]",200,2.344016,4.922197,"(0.25, 0.35]"
92,0.999961,0.336284,0.123077,0.102564,0.437870,150,"[-13, 14]",200,"[-13, 11]",200,3.569949,4.924107,"(0.35, 0.45]"
53,0.999979,0.419616,0.133333,0.051282,0.524506,150,"[-15, 16]",200,"[-15, 13]",200,3.513271,4.912932,"(0.45, 0.55]"
133,0.999976,0.457033,0.133333,0.153846,0.628664,150,"[-15, 16]",200,"[-13, 13]",200,2.361343,4.901708,"(0.55, 0.65]"
94,0.999971,0.531675,0.143590,0.102564,0.722958,150,"[-15, 16]",200,"[-15, 13]",200,3.516279,4.953061,"(0.65, 0.75]"
15,0.999984,0.627070,0.153846,0.000000,0.843608,150,"[-15, 16]",200,"[-17, 18]",200,2.337455,4.858165,"(0.75, 0.85]"
95,0.999971,0.640549,0.153846,0.102564,0.923404,150,"[-16, 17]",200,"[-16, 14]",200,2.348309,4.931543,"(0.85, 0.95]"


In [38]:
# gamma,sqz = 0.152, 0.152

# N = 200

# innercube, nmean= innercubic(gamma, sqz, N)

# print(nmean, innercube.norm())

# wlnnum2, wignormnum2, time12, time22, x3bound, p3bound = wln(innercube, 1e-3, 200, 200, [-10,10], [-10,10])

# print(time12, time22)

# print(wlnnum2, wignormnum2)

In [39]:
# fig2, ax2 = myplotwigner(innercube, x3bound, p3bound)
# plt.show()

In [14]:
# rs = np.unique(np.array(df1.r))

# WLNarr1 = []
# nmeanarr1 = []
# for r in rs:
# #     WLNarr1.append(np.array(df1[df1.r > r-0.01][df1.r < r + 0.01].WLN))
#     WLNarr1.append(np.array(df1[df1.r == r].WLN))
#     nmeanarr1.append(np.array(df1[df1.r == r].nmean))

In [15]:
# rs = np.unique(np.array(df2.r))

# WLNarr2 = []
# for r in rs:
#     WLNarr2.append(np.array(df2[df2.r == r].WLN))

In [16]:
# WLNarr1 =np.array(WLNarr1)
# WLNarr2 =np.array(WLNarr2)
# nmeanarr1 = np.array(nmeanarr1)
# nmeanarr2 = np.array(nmeanarr2)